# MuMoT Demonstration Notebook: Variance-Suppression through Negative Feedback <a class="tocSkip">

## Multiscale Modelling Tool <a class="tocSkip">

*Andreagiovanni Reina, Department of Computer Science, University of Sheffield;
James A. R. Marshall, Department of Computer Science, University of Sheffield*

# Introduction
    
This notebook reproduces results representative of those presented in Reina &amp; Marshall ([2020](#references)), in which it is shown that negative feedback can acts as a mechanism to suppress stochastic fluctuations in a simple collective foraging model.

In [ ]:
import mumot

# Model Definitions

We start with a full model of collective foraging that includes positive feedback, through recruitment, and negative, inhibitory feedback.
Taking collective foraging by honeybees as an exemplar, positive feedback through recruitment occurs via the <i>waggle dance</i> (von Frisch, [1967](#references)), while negative feedback occurs via the <i>stop signal</i> (Nieh, [1993](#references)).

In [ ]:
%%model
$
U -> A : q_A
U -> B : q_B
A -> U : a
B -> U : a
A + U -> A + A : r_a
B + U -> B + B : r_b
A + A -> A + U : z
B + B -> B + U : z
$

In [ ]:
fullmodel = mumot.parseModel(In[2])
fullmodel.show()

We next set a finite system size $N$, since on a short timescale there is a constant pool of foragers. This reduces the system to two ODEs:

In [ ]:
fullmodel = fullmodel.substitute('U = N - A - B')
fullmodel.showODEs()

Next we 'turn-off' the negative social feedback part of the model ($i.e.$ parameter $z=0$) to give a foraging model with only quality-independent positive feedback *r*. This is our **model without negative social feedback**:

In [ ]:
posmodel = fullmodel.substitute('z = 0, r_a = r, r_b = r')
posmodel.showODEs()

Instead, the **model with negative social feedback** has $z>0$ and quality-dependent positive social feedback (with strength $\rho$):

In [ ]:
negmodel = fullmodel.substitute('r_a = \\rho q_A, r_b = \\rho q_B')
negmodel.showODEs()

# Results

In the following, we analyse the models using both infinite-population ODEs, and finite population stochastic simulations.

## Model without negative social feedback

The analysis of the deterministic ODE system predicts that a population starting from a fully uncommmitted initial state ($i.e.$ all individuals at time $t=0$ are in uncommitted state $U$) will converge to the distribution:

$ A = q_A / (q_A + q_B) $

$ B = q_B / (q_A + q_B) $

In [ ]:
int1 = posmodel.integrate(initialState = {'B': 0.0, 'U': 1.0, 'A': 0.0},
                          initWidgets={'a':[0.001, 0, 1, 0.001],'r':[100, 0, 200, 1],
                                      'q_{A}':[0.8, 0, 1, 0.05],'q_{B}':[0.4, 0, 1, 0.05],
                                      'maxTime':[0.5,0.1,2,0.1]},
                          choose_yrange=[0,1], plotProportions=True, ylab="Subpopulations $x_i$")

## Model with negative social feedback

Also for the case of the system with negative social feedback, the analysis of the deterministic ODE system predicts that a population starting from a fully uncommmitted initial state ($i.e.$ all individuals at time $t=0$ are in uncommitted state $U$) will converge to the distribution with a larger convergence time:

$ A = q_A / (q_A + q_B) $

$ B = q_B / (q_A + q_B) $

In [ ]:
int2 = negmodel.integrate(initialState = {'B': 0.0, 'U': 1.0, 'A': 0.0},
                          initWidgets={'a':[0.001, 0, 1, 0.001],'\\rho':[200, 0, 400, 1],
                                      'q_{A}':[0.8, 0, 1, 0.05],'q_{B}':[0.4, 0, 1, 0.05],
                                      'z':[3.4, 0, 50, 0.02],'maxTime':[10,1,1000,1]},
                          choose_yrange=[0,1], plotProportions=True, ylab="Subpopulations $x_i$")

## Noise and variance

By introducing finite-system noise (via SSA), the analysis shows that the model **without** negative social feedback has a large variance around the dynamics predicted by the ODEs

In [ ]:
mc1 = mumot.MuMoTmultiController([posmodel.SSA(silent=True), posmodel.integrate(silent=True)],
                           shareAxes=True,
                           initialState = {'B': 0.0, 'U': 1.0, 'A': 0.0}, 
                           initWidgets={'a':[0.001, 0, 1, 0.001],'r':[100, 0, 200, 1],
                                       'q_{A}':[0.8, 0, 1, 0.05],'q_{B}':[0.4, 0, 1, 0.05],
                                       'maxTime':[2,0.1,2,0.1],'systemSize':[200,10,1000,10],
                                       'runs':[50,1,100,1]},
                           choose_yrange=[0,1], ylab="Subpopulations $x_i$")

Instead, the model **with** negative social feedback has a very small variance around the dynamics predicted by the ODEs

In [ ]:
mc2 = mumot.MuMoTmultiController([negmodel.SSA(silent=True), negmodel.integrate(silent=True)],
                           shareAxes=True,
                           initialState = {'B': 0.0, 'U': 1.0, 'A': 0.0}, 
                           initWidgets={'a':[0.001, 0, 1, 0.001],'rho':[200, 0, 400, 1],
                                      'q_{A}':[0.8, 0, 1, 0.05],'q_{B}':[0.4, 0, 1, 0.05],
                                      'z':[3.4, 0, 50, 0.02],'maxTime':[10,1,1000,1],
                                      'systemSize':[200,10,1000,10],'runs':[10,1,100,1]},
                           choose_yrange=[0,1], ylab="Subpopulations $x_i$")

Returning to the analysis of the ODE system, we see that the model without negative social feedback has very slow dynamics if the initial point (at $t=0$) is different from $U=1$, which can be caused by initial random fluctuations:

In [ ]:
int3 = posmodel.integrate(initWidgets={'initialState':{'B':[0.1,0,1,0.05], 'U':[0.8,0,1,0.05], 'A':[0.1,0,1,0.05]},
                                       'a':[0.001, 0, 1, 0.001],'r':[100, 0, 200, 1],
                                       'q_{A}':[0.8, 0, 1, 0.05],'q_{B}':[0.4, 0, 1, 0.05],
                                       'maxTime':[300000,1,1000000,1]},
                          choose_yrange=[0,1], plotProportions=True, ylab="Subpopulations $x_i$")

Instead the system with negative social feedback has dynamics with speed independed from the initial starting point.

In [ ]:
int4 = negmodel.integrate(initWidgets={'initialState':{'B':[0.1,0,1,0.05], 'U':[0.8,0,1,0.05], 'A':[0.1,0,1,0.05]},
                                       'a':[0.001, 0, 1, 0.001],'\\rho':[200, 0, 400, 1],
                                       'q_{A}':[0.8, 0, 1, 0.05],'q_{B}':[0.4, 0, 1, 0.05],
                                       'z':[3.4, 0, 50, 0.02],'maxTime':[5,1,100,1]},
                          choose_yrange=[0,1], plotProportions=True, ylab="Subpopulations $x_i$")

# References 
<a id='references'></a>
<ul>
<li>Reina, A. &amp; Marshall, J. A. R. (2020) <a href="https://doi.org/10.1101/2020.04.21.053074">Negative feedback may suppress variation to improve collective foraging performance</a>. <i>bioRxiv</i> 2020.04.21.053074. 
<li>von Frisch K (1967). <i>The Dance Language and Orientation of Bees</i>. Belknap Press, Cambridge, MA.
<li>Nieh, J. (1993). The stop signal of the honey bee: reconsidering its message. <i>Behav. Ecol. Sociobiol.</i> <b>33</b>, 51–56.
</ul>